In [ ]:
import openai
import pandas as pd
from flask import Flask, request, jsonify
from langchain.chains import SelfQueryRetriever
from langchain.prompts import PromptTemplate


openai.api_key = 'your_openai_api_key'


prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Find the best matching movie for: {query}"
)


retriever = SelfQueryRetriever(
    prompt_template=prompt_template,
    openai_model="text-davinci-002"
)


def load_movie_data():
    movies = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False)
    ratings = pd.read_csv('title.ratings.tsv.gz', sep='\t', low_memory=False)
    movie_data = pd.merge(movies, ratings, on='tconst', how='inner')
    return movie_data

movie_data = load_movie_data()


def generate_movie_response(query):
    response = retriever(query)
    movie_list = []  

    results = []
    for movie in movie_list:
        title = movie.get('primaryTitle')
        thumbnail_url = movie.get('thumbnail_url', 'default_thumbnail.jpg')
        movie_url = f"https://www.imdb.com/title/{movie.get('tconst')}"
        
        results.append({
            "title": title,
            "thumbnail_url": thumbnail_url,
            "movie_url": movie_url
        })
    return results

# Flask app
app = Flask(__name__)

@app.route('/search', methods=['POST'])
def search_movies():
    query = request.json.get('query')
    results = generate_movie_response(query)
    return jsonify(results)

if __name__ == '__main__':
    app.run(debug=True)

